<a href="https://colab.research.google.com/github/yourusername/yourrepo/blob/main/Score_Evaluation_Model_WandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Score Evaluation Model with W&B Integration

In [ ]:
!pip install transformers datasets evaluate pandas scikit-learn matplotlib seaborn wandb --quiet

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import wandb
import numpy as np
from google.colab import files

In [ ]:
wandb.login()

wandb.init(project="score-evaluation-model", config={
    "model_type": "distilbert-base-uncased",
    "batch_size": 8,
    "epochs": 4,
    "learning_rate": 2e-5,
    "test_size": 0.1
})

In [ ]:
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0]

wandb.log({
    "dataset_size": len(df),
    "score_distribution": wandb.plot.histogram(wandb.Table(data=df, columns=["score"]), 
    "sample_data": wandb.Table(dataframe=df.head())
})

def format_row(row):
    return f"[QUESTION] {row['question']} [IDEAL] {row['ideal_answer']} [USER] {row['user_answer']}"

df['input_text'] = df.apply(format_row, axis=1)
df['label'] = df['score'].astype(float)

dataset = Dataset.from_pandas(df[['input_text', 'label']])
dataset = dataset.train_test_split(test_size=wandb.config.test_size, seed=42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(wandb.config.model_type)

def tokenize(example):
    return tokenizer(example['input_text'], truncation=True, padding="max_length")

tokenized = dataset.map(tokenize)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    wandb.config.model_type, 
    num_labels=1
)

In [ ]:
def compute_metrics(p):
    preds = p.predictions.flatten()
    labels = p.label_ids
    
    mse = mean_squared_error(labels, preds)
    rmse = np.sqrt(mse)
    
    wandb.log({"eval_mse": mse, "eval_rmse": rmse})
    
    return {"mse": mse, "rmse": rmse}

args = TrainingArguments(
    output_dir="score_model",
    per_device_train_batch_size=wandb.config.batch_size,
    per_device_eval_batch_size=wandb.config.batch_size,
    num_train_epochs=wandb.config.epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized["test"])
preds = predictions.predictions.flatten()
labels = predictions.label_ids

plt.figure(figsize=(8, 6))
sns.scatterplot(x=labels, y=preds, alpha=0.6)
plt.xlabel("True Score")
plt.ylabel("Predicted Score")
plt.title("True vs Predicted Scores on Validation Set")
plt.grid(True)

wandb.log({"true_vs_predicted": wandb.Image(plt)})
plt.show()

wandb.log({
    "predictions": wandb.Table(
        columns=["True Score", "Predicted Score"],
        data=list(zip(labels.tolist(), preds.tolist()))
})

In [ ]:
trainer.save_model("scoring_distilbert_model")
tokenizer.save_pretrained("scoring_distilbert_model")

!zip -r scoring_model.zip scoring_distilbert_model

artifact = wandb.Artifact('scoring_model', type='model')
artifact.add_file("scoring_model.zip")
wandb.log_artifact(artifact)

wandb.finish()